In [1]:
from probability import normal_cdf, inverse_normal_cdf
# 이전장에서 작업한 확률 관련 코드 불러옴
import math, random

In [6]:
def normal_approximation_to_binomial(n, p):
    """Binomial(n, p)에 해당하는 mu(평균)와 sigma(표준편차) 계산"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [86]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000,0.5)
mu_0, sigma_0

(500.0, 15.811388300841896)

In [12]:
from probability import normal_cdf, inverse_normal_cdf
# 누적분포함수는 확률변수가 특정 값보다 작을 확률을 나타냄
normal_probability_below = normal_cdf

# 만약 확률변수가 특정 값보다 작지 않다면 특정 값보다 크다는 것을 의미 함
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)
    
# 만약 확률변수가 hi보다 작고 lo보다 작지 않다면, 확률변수는 hi와 lo 사이에 존재함
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 범위 밖에 존재한다면, 범위 안에 존재하지 않는다는 것을 의미함.
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

def normal_upper_bound(probability, mu=0, sigma=1):
    """P(Z <= z)=probability.py인 z값을 반환"""
    return inverse_normal_cdf(probability, mu, sigma)
    
def normal_lower_bound(probability, mu=0, sigma=1):
    """P(Z >= z)=probability.py인 z값을 반환"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """입력한 probability 값을 포함하고,
    평균을 중심으로 대칭적인 구간을 반환"""
    tail_probability = (1 - probability) / 2

    # 구간의 상한은 tail_probability 값 이상의 확률 값을 갖고 있다.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # 구간의 하한은 tail_probability 값 이하의 확률 값을 갖고 있다.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [67]:
# p가 0.5라고 가정할 때, 유의수준이 5%인 구간
lo,hi = normal_two_sided_bounds(0.95, mu_0, sigma_0) 
print(lo,hi)
# p = 0.55인 경우의 실제 평균과 표준 편차
mu_1, sigma_1 = normal_approximation_to_binomial(3000,0.55)
print(mu_1,sigma_1)

1446.3242069002204 1553.6757930997796
1650.0000000000002 27.248853186877426


In [60]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(1446.3242069002204, 1553.6757930997796)

In [68]:
# 제2종 오류는 귀무가설을 기각하지 못한다는 의미함
# 즉, X가 주어진 구간 안에 존재할 경우를 의미함
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
print(type_2_probability)
power = 1 - type_2_probability
print(power)

0.0002039014297088304
0.9997960985702912


In [69]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
type_2_probability=normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.9999413454954649

In [40]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is above x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is below x
        return 2 * normal_probability_below(x, mu, sigma)   

In [97]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

In [89]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [83]:
def count_extreme_values():
    extreme_value_count = 0
    for _ in range(100000):
        num_heads = sum(1 if random.random() < 0.5 else 0    # 앞면이 나온 경우
                        for _ in range(1000))                # 동전 1000번 던
        if num_heads >= 530 or num_heads <= 470:             # 극한 값이
            extreme_value_count += 1                         # 몇 번 나오는지

    return extreme_value_count / 100000

In [84]:
count_extreme_values()

0.06202

In [10]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

True

In [11]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

False

In [93]:
upper_p_value = normal_probability_above
t1=upper_p_value(524.5,mu_0,sigma_0)
t2=upper_p_value(526.5,mu_0,sigma_0)
t1, t2

(0.06062885772582083, 0.04686839508859242)

In [13]:
def run_experiment():
    """flip a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [14]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]

[sum(experiment) for experiment in experiments][:10]

[508, 491, 484, 483, 502, 499, 483, 511, 481, 538]

In [15]:
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])
num_rejections

46

In [16]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [98]:
def B(alpha, beta):
    """모든 확률 값의 합이 1이 되도록 해주는 정규화 값"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # [0, 1] 구간 밖에서는 밀도가 없음
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

In [18]:
z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [19]:
two_sided_p_value(z) # 두 광고 효과의 평균이 같을 확률이 높음 (p > 0.05) = 광고효과가 같음

0.254141976542236

In [20]:
z = a_b_test_statistic(1000, 200, 1000, 150)
z

-2.948839123097944

In [21]:
two_sided_p_value(z) # 두 광고 효과의 평균이 같을 확률이 매우 낮음 (p < 0.05) = 광고효과가 다름

0.003189699706216853

In [99]:
##
#
# Bayesian Inference
#
##

def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # no weight outside of [0, 1]    
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)